In [ ]:
import re

# ---------------------------------------------------------
# 0. 전처리 모듈 개요 (ver5 – rule-based + 의료 의미 보호)
# ---------------------------------------------------------
# 이 모듈은 KCD/SNOMED 용어를 동일한 규칙(v1.5)에 따라 전처리하기 위한 공통 함수 집합입니다.
# 목표:
#   - KCD/SNOMED 모두에 대해 일관된 정규화 규칙 유지
#   - 매핑에 중요한 의료 의미(benign/malignant, acute/chronic, due to, etc.)는 최대한 보존
#   - 불필요한 메타 표현만 제거 (NOS, otherwise specified, with/without cavitation 등)
#   - 이후 모델링/재랭킹 단계에서 활용할 수 있도록 modifier 정보도 함께 제공


# ---------------------------------------------------------
# 1. 설정: 불용어 / 기능어 / 예외 단어 / modifier 사전
# ---------------------------------------------------------

# 1-1. 괄호 태그 중 "완전 메타"로 간주해서 버릴 것들 (SNOMED FSN 태그 등)
GENERIC_PAREN_TAGS = {
    "disorder",
    "finding",
    "procedure",
    "body structure",
    "morphologic abnormality",
    "situation",
    "event",
    "observable entity",
    "organism",
    "substance",
    "pharmaceutical / biologic product",
    "qualifier value",
    "physical object",
    "environments and geographical locations",
}

# 1-2. 도메인 공통 불용어 (의미적 impact가 상대적으로 낮은 메타 표현)
# 👉 ver5에서는 의미를 많이 훼손하는 단어("other", "due to", "associated with" 등)는 STOPWORDS에서 제거
STOPWORDS = [
    # NOS / specified / unspecified 계열: 너무 자주 나오고 의미가 약한 메타 정보
    "not otherwise specified",
    "otherwise specified",
    "other and unspecified",
    "other and unspecified parts of",
    "other and ill-defined parts of",
    "ill-defined",
    "unspecified part of",
    "other parts of",
    "other unspecified",
    "other forms of",
    "unspecified side",
    "nos",

    # 위치/부위 표현 중 메타적인 것들
    "part of",
    "parts of",
    "site of",
    "portion of",
    "sites",
    "other sites",

    # 확인/검사 방식 관련 메타 표현
    "confirmed by",
    "confirmed only",
    "confirmed histologically",
    "confirmed bacteriologically",

    # 진행 양상 메타 표현
    "with cavitation",
    "without cavitation",
    "with or without",
]

# ⚠️ 주의: 아래 단어/구는 STOPWORDS에서 *제거*했습니다 (의미 보존을 위해).
#   - "other" (단독)
#   - "specified"
#   - "unspecified"
#   - "due to"
#   - "associated with"
#   - "resulting in"
#   → 이들은 나중에 rule-based rerank에서 modifier로 활용할 수 있도록 그대로 두는 것이 좋음.

# 1-3. 기능어(function words) – query relaxation 단계에서 선택적 제거용
FUNCTION_WORDS = ["of", "the", "for", "in", "a", "an"]

# 1-4. 복수형 예외 단어 목록 (s 제거하면 안 되는 단어들)
EXCEPTION_WORDS = [
    # 감염성 질환류
    "tuberculosis",
    "measles",
    "mumps",
    "rabies",
    "pertussis",
    "rubella",
    "salmonellosis",
    "shigellosis",
    "amoebiasis",
    "giardiasis",
    "cryptosporidiosis",
    "isosporiasis",
    "balantidiasis",
    "trichomoniasis",
    "coccidiosis",

    # 염증/질환 계열
    "enteritis",
    "colitis",
    "gastritis",
    "pneumonitis",
    "hepatitis",
    "dermatitis",
    "encephalitis",
    "lymphadenitis",
    "appendicitis",

    # 기타 해부/질환 용어
    "abscess",
    "sinus",
    "sepsis",
    "anus",
    "bronchus",
    "thymus",
    "lens",
    "iris",
    "mass",
    "gas",
    "stress",

    # 추가 보호 단어 (ver5 보강)
    "virus",
    "fungus",
    "fungi",
    "coccus",
    "cocci",
    "bacterium",
    "bacteria",
]

# 1-5. 패턴 기반 예외 suffix
EXCEPTION_SUFFIXES = ["itis", "osis", "iasis", "esis", "us"]

# 1-6. 의학적 modifier 사전 (나중에 rule-based rerank에서 사용할 수 있도록 추출용)
MODIFIER_PATTERNS = {
    "behavior": [
        "benign",
        "malignant",
        "in situ",
        "unspecified",
    ],
    "temporal": [
        "acute",
        "subacute",
        "chronic",
        "recurrent",
        "persistent",
    ],
    "relation": [
        "due to",
        "associated with",
        "resulting in",
        "secondary to",
        "following",
    ],
    "extent": [
        "local",
        "localized",
        "generalized",
        "systemic",
        "diffuse",
        "focal",
    ],
    "laterality": [
        "left",
        "right",
        "bilateral",
    ],
}


# ---------------------------------------------------------
# 2. 기본 텍스트 정규화 (v1.5)
# ---------------------------------------------------------

def _strip_generic_parentheses(text: str) -> str:
    """
    괄호 처리 규칙 (ver5):
      - 괄호 안 내용이 GENERIC_PAREN_TAGS에 해당하면 통째로 제거
      - 그렇지 않으면 괄호만 제거하고 내용은 살려서 문자열에 포함
        예: "pyelonephritis (acute)" → "pyelonephritis acute"
    """
    def repl(m: re.Match) -> str:
        inner = m.group(1).strip()
        inner_lower = inner.lower()
        if inner_lower in GENERIC_PAREN_TAGS:
            return " "  # 완전 메타 태그: 내용도 제거
        else:
            return f" {inner} "  # 괄호만 제거하고 내용은 유지

    return re.sub(r"\(([^)]*)\)", repl, text)


def normalize_text(text: str) -> str:
    """
    v5.1 규칙:
      - 소문자 변환 제거 (의료 도메인 의미 보호)
      - 괄호 내용 처리
      - 불필요한 기호/공백 정리
    """
    if not isinstance(text, str):
        return ""

    # 0) 양쪽 공백 제거
    text = text.strip()

    # 1) 괄호: FSN 메타 태그만 제거, 의미 있는 내용 유지
    text = _strip_generic_parentheses(text)

    # 2) 줄바꿈/탭 → 공백
    text = re.sub(r"[\n\t]", " ", text)

    # 3) 특수문자 처리
    text = text.replace(",", " ").replace(";", " ").replace(":", " ")
    text = text.replace("'", " ")

    # 4) 하이픈: a-b → a b (대소문자 모두 고려)
    text = re.sub(r"([A-Za-z])\-([A-Za-z])", r"\1 \2", text)

    # 5) 중복 공백 정리
    text = re.sub(r"\s+", " ", text).strip()

    return text


# ---------------------------------------------------------
# 3. 도메인 불용어 제거 (ver5 – 완화 버전)
# ---------------------------------------------------------

def remove_stopwords(text: str) -> str:
    """
    ver5 불용어 제거:
      - STOPWORDS에 정의된 phrase/단어를 제거
      - 긴 phrase부터 제거하기 위해 길이 순으로 내림차순 정렬
      - 'due to', 'associated with', 'resulting in', 'other' 등은 *제거하지 않음*
    """
    out = text
    for sw in sorted(STOPWORDS, key=len, reverse=True):
        pattern = rf"\b{re.escape(sw)}\b"
        out = re.sub(pattern, " ", out)
    out = re.sub(r"\s+", " ", out).strip()
    return out


# ---------------------------------------------------------
# 4. 복수형 → 단수형 정규화 (3번 규칙: 보수적 적용)
# ---------------------------------------------------------

def normalize_plural_word(word: str) -> str:
    """
    - 예외 단어(EXCEPTION_WORDS)는 그대로 유지
    - 예외 suffix(EXCEPTION_SUFFIXES)로 끝나면 그대로 유지 (itis, osis, iasis, esis, us 등)
    - 그 외, 길이 > 3 이면서 s로 끝나는 단어는 s 제거
      (ex: lungs -> lung, kidneys -> kidney)
    """
    if word in EXCEPTION_WORDS:
        return word

    for suf in EXCEPTION_SUFFIXES:
        if word.endswith(suf):
            return word

    if word.endswith("s") and len(word) > 3:
        return word[:-1]

    return word


def normalize_plural(text: str) -> str:
    tokens = text.split()
    tokens = [normalize_plural_word(t) for t in tokens]
    return " ".join(tokens)


# ---------------------------------------------------------
# 5. 해부학/병리 분리 (4번 규칙)
# ---------------------------------------------------------

def split_pathology_and_anatomy(text: str):
    """
    4-1, 4-2 규칙:
      - 첫 번째 ' of '를 기준으로 앞: pathology, 뒤: anatomy
      - 예: 'carcinoma in situ of anus and anal canal'
        → pathology: 'carcinoma in situ'
        → anatomy: 'anus and anal canal'
    """
    if " of " not in text:
        return text.strip(), ""

    parts = text.split(" of ", 1)
    pathology = parts[0].strip()
    anatomy = parts[1].strip()
    return pathology, anatomy


def split_anatomy_list(anatomy_str: str):
    """
    anatomy 영역을 ','와 'and'를 기준으로 나누어 리스트로 반환.
    예: 'anus and anal canal' -> ['anus', 'anal canal']
    """
    if not anatomy_str:
        return []

    tmp = anatomy_str.replace(",", " and ")
    parts = [p.strip() for p in tmp.split(" and ") if p.strip()]
    return parts


# ---------------------------------------------------------
# 6. Query Relaxation용 쿼리 후보 생성 (6번 규칙)
# ---------------------------------------------------------

def generate_query_candidates(source_term: str):
    """
    Query Relaxation 단계:
      - Step 0: normalize_text
      - Step 1: + stopword 제거
      - Step 2: + 복수형 → 단수형
    (function words(of, in, for, the, a, an) 제거 단계는 별도 로직에서 필요 시 추가)
    """
    step0 = normalize_text(source_term)
    step1 = remove_stopwords(step0)
    step2 = normalize_plural(step1)
    return [step0, step1, step2]


# ---------------------------------------------------------
# 7. 의학적 modifier 추출 (rule-based)
# ---------------------------------------------------------

def extract_modifiers(text: str):
    """
    원문(또는 normalize 후 텍스트)에서 의학적 modifier(acute, chronic, benign, malignant, due to 등)를
    rule-based로 추출하여 dict 형태로 반환.

    반환 예:
      {
        "behavior": ["benign"],
        "temporal": ["acute"],
        "relation": ["due to"],
        ...
      }
    """
    if not isinstance(text, str):
        return {}

    t = text.lower()
    found = {}

    for cat, patterns in MODIFIER_PATTERNS.items():
        hits = []
        for p in patterns:
            # phrase 단위 매칭 (공백 포함)
            pattern = rf"\b{re.escape(p)}\b"
            if re.search(pattern, t):
                hits.append(p)
        if hits:
            found[cat] = sorted(set(hits))

    return found


# ---------------------------------------------------------
# 8. 전체 전처리 파이프라인 (KCD/SNOMED 용어 1개 단위)
# ---------------------------------------------------------

def preprocess_kcd_term(source_term: str):
    """
    KCD 영문 용어 하나에 대해 full pipeline (ver5):
      - normalized: 기본 정규화(소문자, 괄호 처리, 구두점/공백 정리)
      - no_stopwords: 도메인 불용어 제거
      - no_plural: 복수형 정규화
      - pathology / anatomy_list 분리 (4번 규칙)
      - query_candidates: Step0~2 쿼리 문자열
      - modifiers: behavior/temporal/relation/extent/laterality 등의 modifier 추출 결과
    """
    normalized = normalize_text(source_term)
    no_sw = remove_stopwords(normalized)
    no_plural = normalize_plural(no_sw)

    pathology, anatomy = split_pathology_and_anatomy(no_plural)
    anatomy_list = split_anatomy_list(anatomy)

    modifiers = extract_modifiers(source_term)

    return {
        "normalized": normalized,
        "no_stopwords": no_sw,
        "no_plural": no_plural,
        "pathology": pathology,
        "anatomy_list": anatomy_list,
        "query_candidates": generate_query_candidates(source_term),
        "modifiers": modifiers,
    }